# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# instantiate a Spark session 
ss = SparkSession \
	.builder \
	.appName("SparkSQL") \
	.getOrCreate()

path = "data/sparkify_log_small.json"
user_log = ss.read.json(path)

# create a view to use with the SQL queries
user_log.createOrReplaceTempView("user_log_table")


24/01/26 12:49:27 WARN Utils: Your hostname, geoffroy-XPS-15-9520 resolves to a loopback address: 127.0.1.1; using 192.168.13.105 instead (on interface wlp0s20f3)
24/01/26 12:49:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/26 12:49:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Question 1

Which page did user id ""(empty string) NOT visit?

In [2]:
ss.sql('''
	SELECT all_pages.page
	FROM (
		SELECT DISTINCT page
		FROM user_log_table
	) AS all_pages
	LEFT ANTI JOIN (
		SELECT DISTINCT page
		FROM user_log_table
		WHERE userId = ""
	) AS blank_pages
	ON all_pages.page = blank_pages.page;	
''').show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+


# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [4]:
ss.sql('''
	SELECT COUNT(DISTINCT userId)
	FROM user_log_table
	WHERE gender = 'F'
'''
).show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+


# Question 4

How many songs were played from the most played artist?

In [5]:
ss.sql('''
	SELECT artist, COUNT(artist) AS plays
	FROM user_log_table
	WHERE page = 'NextSong'
	GROUP BY artist
	ORDER BY plays DESC
	LIMIT 1
'''
).show()

+--------+-----+
|  artist|plays|
+--------+-----+
|Coldplay|   83|
+--------+-----+


# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [13]:
# SELECT CASE WHEN 1 > 0 THEN 1 WHEN 2 > 0 THEN 2.0 ELSE 1.2 END;
is_home = ss.sql("""
	SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home 
	FROM user_log_table 
	WHERE (page = 'NextSong') or (page = 'Home') \
""")

# keep the results in a new view
is_home.createOrReplaceTempView("is_home_table")

# find the cumulative sum over the is_home column
cumulative_sum = ss.sql("""
	SELECT *, SUM(is_home) OVER 
    (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period 
    FROM is_home_table
    """)

# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_table")

# find the average count for NextSong
ss.sql("""
	SELECT AVG(count_results) 
	FROM 
		(SELECT COUNT(*) AS count_results 
		FROM period_table 
		GROUP BY userID, period, page HAVING page = 'NextSong') AS counts""").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+


### Lets analyse step by step the process used in the solution above

#### Step 1: is_home
Add a column `is_home` to the `user_log_table` table that has a value of `1` when the `page` is `Home` and `0` otherwise

In [16]:
ss.sql("""SELECT *
	   FROM is_home_table \
            WHERE userID = 100 \
            """).show()

+------+--------+-------------+-------+
|userID|    page|           ts|is_home|
+------+--------+-------------+-------+
|   100|NextSong|1513750214284|      0|
|   100|NextSong|1513750442284|      0|
|   100|    Home|1513775431284|      1|
|   100|NextSong|1513775556284|      0|
|   100|NextSong|1513775710284|      0|
|   100|    Home|1513776194284|      1|
|   100|NextSong|1513776308284|      0|
|   100|    Home|1513839673284|      1|
+------+--------+-------------+-------+


#### Step 2: cumulative_sum
A column, period, is created to cumulatively sum the homevisit flags within each user's partition, effectively segmenting the data into periods based on home page visits. Each period represents the time from one home page visit to the next for a given user.

In [18]:
ss.sql('''
	SELECT *
	FROM period_table
	WHERE userID = 100
''').show()

+------+--------+-------------+-------+------+
|userID|    page|           ts|is_home|period|
+------+--------+-------------+-------+------+
|   100|    Home|1513839673284|      1|     1|
|   100|NextSong|1513776308284|      0|     1|
|   100|    Home|1513776194284|      1|     2|
|   100|NextSong|1513775710284|      0|     2|
|   100|NextSong|1513775556284|      0|     2|
|   100|    Home|1513775431284|      1|     3|
|   100|NextSong|1513750442284|      0|     3|
|   100|NextSong|1513750214284|      0|     3|
+------+--------+-------------+-------+------+
